In [1]:
import os
import textwrap

import langchain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai.model_garden import ChatAnthropicVertex
from langchain_openai import ChatOpenAI

import llm_cache_stats_wrapper
import simple_llm_cache

# In order to make it easy to run work projects and personal AI experiments, override these key values with the value of *_PERSONAL, if set.
if "OPENAI_API_KEY_PERSONAL" in os.environ:
    print("Using key from OPENAI_API_KEY_PERSONAL environment variable")
    os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY_PERSONAL"]
if "ANTHROPIC_API_KEY_PERSONAL" in os.environ:
    print("Using key from ANTHROPIC_API_KEY_PERSONAL environment variable")
    os.environ["ANTHROPIC_API_KEY"] = os.environ["ANTHROPIC_API_KEY_PERSONAL"]
if "GOOGLE_APPLICATION_CREDENTIALS_PERSONAL" in os.environ:
    print("Using key from GOOGLE_APPLICATION_CREDENTIALS_PERSONAL environment variable")
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.environ["GOOGLE_APPLICATION_CREDENTIALS_PERSONAL"]

verbose = False
temperature = 0.5
vertex_model_name_prefix = "vertexai-"

langchain.llm_cache = llm_cache_stats_wrapper.LlmCacheStatsWrapper(simple_llm_cache.SimpleLlmCache("llm-cache.json"))

def dump_cache_stats_since_last_call():
    print(langchain.llm_cache.get_cache_stats_summary())
    langchain.llm_cache.clear_cache_stats()

template = """Answer the following question as if you are a {character} character:
{question}
"""

prompt = PromptTemplate(
    input_variables=["character", "question"],
    template=template)

for model_name in [
    # "gpt-3.5-turbo",
    # "gpt-4",
    # "gpt-4o-mini",
    "gpt-4o",
    # "claude-3-haiku-20240307",
    "gemini-1.5-flash-001",
    "vertexai-claude-3-haiku@20240307",
]:
    if model_name.startswith("gpt-"):
        llm = ChatOpenAI(
            temperature=temperature,
            model_name = model_name)
    elif model_name.startswith("claude-"):
        llm = ChatAnthropic(
            temperature=temperature,
            model_name = model_name)
    elif model_name.startswith("gemini-"):
        llm = ChatVertexAI(temperature=temperature, model_name=model_name)
    elif model_name.startswith(vertex_model_name_prefix):
        vertex_model_name = model_name[len(vertex_model_name_prefix) :]
        llm = ChatAnthropicVertex(
            temperature=temperature, model_name=vertex_model_name
        )

    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose)

    for trial in range(2):
        print(f"\n*** {model_name} trial {trial} ***")
        langchain.llm_cache.inner_cache.set_trial(trial)
        output = chain.predict(
            character="pirate",
            question="What is a neural network?")
        print(textwrap.fill(output, width=80))

print()
dump_cache_stats_since_last_call()

Using key from OPENAI_API_KEY_PERSONAL environment variable
Using key from GOOGLE_APPLICATION_CREDENTIALS_PERSONAL environment variable


C:\Users\forre\AppData\Local\Temp\ipykernel_24028\660374180.py:69: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(
c:\s\MinSampleTemplates\LangChain\simple_llm_cache.py:111: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  generations.append(loads(generation))



*** gpt-4o trial 0 ***
Arrr, ye be askin' about a neural network, eh? Well, let me spin ye a yarn
that'll make it clear as the seven seas. A neural network be a mighty
contraption, much like a crew o' sailors workin' together on a grand ship.   Ye
see, it be made up o' nodes, which be like the crew members. These nodes be
organized in layers, like the decks o' a ship. The first layer be the input
layer, where ye be feedin' in yer data, like provisions loaded onto the ship.
Then, there be hidden layers, where the real work be done, like the crew
scrubbin' the decks and hoistin' the sails. Finally, ye got yer output layer,
where the results be spit out, like treasure found after a long voyage.  Each
node in a layer be connected to nodes in the next layer by weighted lines, like
ropes 'n pulleys. These weights be adjusted durin' trainin', like a crew
learnin' the best way to sail through a storm. The more ye train yer network,
the better it gets at recognizin' patterns and makin' predict

In [2]:
import asyncio


async def predict_async(trial):
    print(f"\n*** Calling apredict trial {trial}")
    langchain.llm_cache.inner_cache.set_trial(trial)
    output = await chain.apredict(character="poet", question="What is a neural network?")
    print(f"\n*** Returned from apredict trial {trial}")
    print(output)
    print()


# Use the async method to demonstrate that the cache also works for async calls
coroutines = []
for trial in range(10):
    coroutine = predict_async(trial)
    coroutines.append(coroutine)

await asyncio.gather(*coroutines)


dump_cache_stats_since_last_call()


*** Calling apredict trial 0

*** Calling apredict trial 1

*** Calling apredict trial 2

*** Calling apredict trial 3

*** Calling apredict trial 4

*** Calling apredict trial 5

*** Calling apredict trial 6

*** Calling apredict trial 7

*** Calling apredict trial 8

*** Calling apredict trial 9

*** Returned from apredict trial 0
Here is a poetic response to the question "What is a neural network?":

A neural network, a tapestry of threads,
Weaving connections, where knowledge is fed.
Neurons, like stars, in a galaxy bright,
Firing and flashing, a dance of insight.

Synapses, bridges, that span the unknown,
Transmitting data, a current has grown.
Layers of logic, a labyrinth of thought,
Patterns emerging, from what has been wrought.

This intricate web, a mind's inner maze,
Learns and adapts, through its neural phase.
A symphony of signals, a cognitive song,
Unraveling mysteries, right and wrong.

A neural network, a marvel to behold,
A model of mind, its wonders untold.
In its neu